In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

C:\Users\aditg\anaconda3\envs\ANN_project_ete\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
model=load_model('model.keras')
with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender=pickle.load(f)
with open('onehot_encoder.pkl', 'rb') as f:
    onehot_encoder=pickle.load(f)
with open('scaler.pkl', 'rb') as f:
    scaler=pickle.load(f)

C:\Users\aditg\anaconda3\envs\ANN_project_ete\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [7]:
geo_encoded=onehot_encoder.transform([[input_data['Geography']]])
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder.get_feature_names_out(['Geography']))
geo_encoded_df

C:\Users\aditg\anaconda3\envs\ANN_project_ete\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [9]:
input_df=pd.DataFrame([input_data])##converting dictionary to dataframe

In [14]:
input_df['Gender']=label_encoder_gender.transform([input_data['Gender']])
input_df

C:\Users\aditg\anaconda3\envs\ANN_project_ete\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [11]:
input_data=pd.concat([input_df.reset_index(drop=True),geo_encoded_df],axis=1)
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [15]:
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [16]:
##Scale the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [17]:
prediction=model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step


In [18]:
prediction

array([[0.03461254]], dtype=float32)

In [19]:
prediction_proba=prediction[0][0]
prediction_proba

np.float32(0.034612536)

In [20]:
if prediction_proba>0.5:
    print(f"The customer is likely to exit with a probability of {prediction_proba}")
else:
    print(f"The customer is not likely to exit with a probability of {1-prediction_proba}")

The customer is not likely to exit with a probability of 0.9653874635696411
